In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, LSTM, CuDNNLSTM, Flatten, Dropout, Reshape, BatchNormalization, PReLU, ELU, LeakyReLU
from keras import optimizers
from keras import initializers
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
def slidingWindow(sequence, labels, winSize, step, noNull):

    # Verify the inputs
    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")
    if winSize > len(sequence):
        raise Exception("**ERROR** winSize must not be larger than sequence length.")
 
    # number of chunks
    numOfChunks = ((len(sequence)-winSize)//step)+1
 
    # Do the work
    for i in range(0,numOfChunks*step,step):
        segment = sequence[i:i+winSize]
        seg_labels = labels[i:i+winSize]
        if noNull:
            if seg_labels[-1] != 0:
                yield segment, seg_labels
        else:
            yield segment, seg_labels

In [3]:
def get_most_frequent(labels):
    
    (values, counts) = np.unique(labels, return_counts=True)
    index = np.argmax(counts)
    return values[index]
    

def segment_data(X_train, y_train, X_val, y_val, X_test, y_test, winSize, step, noNull=False):
    assert len(X_train) == len(y_train)
    assert len(X_val) == len(y_val)
    assert len(X_test) == len(y_test)
    # obtain chunks of data
    train_chunks = slidingWindow(X_train, y_train , winSize, step, noNull)
    val_chunks = slidingWindow(X_val, y_val, winSize, step, noNull)
    test_chunks = slidingWindow(X_test, y_test, winSize, step, noNull)
    
    # segment the data
    train_segments = []
    train_labels = []
    for chunk in train_chunks:
        data = chunk[0]
        labels = chunk[1]
        train_segments.append(data)
        train_labels.append(get_most_frequent(labels))
        
    val_segments = []
    val_labels = []
    for chunk in val_chunks:
        data = chunk[0]
        labels = chunk[1]
        val_segments.append(data)
        val_labels.append(get_most_frequent(labels))
    
    test_segments = []
    test_labels = []
    for chunk in test_chunks:
        data = chunk[0]
        labels = chunk[1]
        test_segments.append(data)
        test_labels.append(get_most_frequent(labels))
        
    return np.array(train_segments), np.array(train_labels), np.array(val_segments), np.array(val_labels), np.array(test_segments), np.array(test_labels)

In [4]:
def prepare_data(train_data, val_data, test_data):

    train_labels = np.array(train_data['labels'].values)
    val_labels = np.array(val_data['labels'].values)
    test_labels = np.array(test_data['labels'].values)
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_data.drop(['labels'], axis=1, inplace=True)
    val_data.drop(['labels'], axis=1, inplace=True)
    test_data.drop(['labels'], axis=1, inplace=True)
    data = pd.concat([train_data,val_data,test_data])
    scaler.fit(data)
    train_data = scaler.transform(train_data)
    val_data = scaler.transform(val_data)
    test_data = scaler.transform(test_data)
    
    return train_data, val_data, test_data, train_labels, val_labels, test_labels

In [5]:
# import train data
adl_1_1 = pd.read_csv("./full_dataset/CIP_interpolation/ADL1Opportunity_taskB2_S1.csv",header=None)
adl_1_2 = pd.read_csv("./full_dataset/CIP_interpolation/ADL2Opportunity_taskB2_S1.csv",header=None)
adl_1_3 = pd.read_csv("./full_dataset/CIP_interpolation/ADL3Opportunity_taskB2_S1.csv",header=None)
adl_1_4 = pd.read_csv("./full_dataset/CIP_interpolation/ADL4Opportunity_taskB2_S1.csv",header=None)
adl_1_5 = pd.read_csv("./full_dataset/CIP_interpolation/ADL5Opportunity_taskB2_S1.csv",header=None)
drill_1 = pd.read_csv("./full_dataset/CIP_interpolation/Drill1Opportunity_taskB2.csv",header=None)
adl_2_1 = pd.read_csv("./full_dataset/CIP_interpolation/ADL1Opportunity_taskB2_S2.csv",header=None)
adl_2_2 = pd.read_csv("./full_dataset/CIP_interpolation/ADL2Opportunity_taskB2_S2.csv",header=None)
drill_2 = pd.read_csv("./full_dataset/CIP_interpolation/Drill2Opportunity_taskB2.csv",header=None)
adl_3_1 = pd.read_csv("./full_dataset/CIP_interpolation/ADL1Opportunity_taskB2_S3.csv",header=None)
adl_3_2 = pd.read_csv("./full_dataset/CIP_interpolation/ADL2Opportunity_taskB2_S3.csv",header=None)
drill_3 = pd.read_csv("./full_dataset/CIP_interpolation/Drill3Opportunity_taskB2.csv",header=None)

# import validation data
adl_2_3 = pd.read_csv("./full_dataset/CIP_interpolation/ADL3Opportunity_taskB2_S2.csv",header=None)
adl_3_3 = pd.read_csv("./full_dataset/CIP_interpolation/ADL3Opportunity_taskB2_S3.csv",header=None)

# import test data
adl_2_4 = pd.read_csv("./full_dataset/CIP_interpolation/ADL4Opportunity_taskB2_S2.csv",header=None)
adl_2_5 = pd.read_csv("./full_dataset/CIP_interpolation/ADL5Opportunity_taskB2_S2.csv",header=None)
adl_3_4 = pd.read_csv("./full_dataset/CIP_interpolation/ADL4Opportunity_taskB2_S3.csv",header=None)
adl_3_5 = pd.read_csv("./full_dataset/CIP_interpolation/ADL5Opportunity_taskB2_S3.csv",header=None)

In [6]:
train_frames = [adl_1_1,adl_1_2,adl_1_3,adl_1_4,adl_1_5,drill_1,adl_2_1,adl_2_2,drill_2,adl_3_1,adl_3_2,drill_3]
val_frames = [adl_2_3,adl_3_3]
test_frames = [adl_2_4,adl_2_5,adl_3_4,adl_3_5]
train_data = pd.concat(train_frames)
val_data = pd.concat(val_frames)
test_data = pd.concat(test_frames)
train_data.rename(columns ={113: 'labels'}, inplace =True)
val_data.rename(columns ={113: 'labels'}, inplace =True)
test_data.rename(columns ={113: 'labels'}, inplace =True)

In [7]:
print("shapes: train {0}, val {1}, test {2}".format(train_data.shape, val_data.shape, test_data.shape))

shapes: train (497014, 114), val (60949, 114), test (118750, 114)


In [8]:
scaled_train, scaled_val, scaled_test, train_labels, val_labels, test_labels = prepare_data(train_data, val_data, test_data)

In [9]:
num_sensors = 113
window_size = 64
step_size = 10
classes = 18

In [10]:
train_segments, train_labels, val_segments, val_labels, test_segments, test_labels = segment_data(scaled_train, train_labels, scaled_val, val_labels,
                                                                                                  scaled_test, test_labels, window_size, step_size)

In [12]:
print(train_labels.shape)

(49696,)


In [13]:
encoder = OneHotEncoder()
train_labels = encoder.fit_transform(train_labels.reshape(-1,1)).toarray()
val_labels = encoder.transform(val_labels.reshape(-1,1)).toarray()
test_labels = encoder.transform(test_labels.reshape(-1,1)).toarray()

In [14]:
# reshape input for CNN
reshaped_train = train_segments.reshape(-1, window_size, num_sensors, 1)
reshaped_val = val_segments.reshape(-1, window_size, num_sensors, 1)
reshaped_test = test_segments.reshape(-1, window_size, num_sensors, 1)

In [20]:
model = Sequential()

kernel_strides = 1
dropout_prob = 0.5
inputshape = (window_size, num_sensors, 1)

In [21]:
model.add(BatchNormalization(input_shape=inputshape))

In [22]:
model.add(Conv2D(50, kernel_size=(11,1), strides=kernel_strides,
                 kernel_initializer='glorot_normal', name='1_conv_layer'))
model.add(ELU())

In [23]:
model.add(MaxPooling2D(pool_size=(2, 1)))

In [24]:
model.add(Reshape((27, 50*num_sensors)))

In [25]:
model.add(CuDNNLSTM(27,kernel_initializer='glorot_normal', return_sequences=True, name='1_lstm_layer'))

model.add(CuDNNLSTM(27,kernel_initializer='glorot_normal',return_sequences=False, name='2_lstm_layer'))

model.add(Dropout(dropout_prob, name='2_dropout_layer'))

model.add(Dense(512, kernel_initializer='glorot_normal',
                bias_initializer=initializers.Constant(value=0.1), name='dense1_layer'))
model.add(ELU())
model.add(Dropout(dropout_prob, name='3_dropout_layer'))

model.add(Dense(classes,kernel_initializer='glorot_normal',
                bias_initializer=initializers.Constant(value=0.1),activation='softmax', name='softmax_layer'))

opt = optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [26]:
for layer in model.layers:
    print(str(layer.name) + ': input shape: ' + str(layer.input_shape) + ' output shape: ' + str(layer.output_shape))

batch_normalization_2: input shape: (None, 64, 113, 1) output shape: (None, 64, 113, 1)
1_conv_layer: input shape: (None, 64, 113, 1) output shape: (None, 54, 113, 50)
elu_2: input shape: (None, 54, 113, 50) output shape: (None, 54, 113, 50)
max_pooling2d_2: input shape: (None, 54, 113, 50) output shape: (None, 27, 113, 50)
reshape_1: input shape: (None, 27, 113, 50) output shape: (None, 27, 5650)
1_lstm_layer: input shape: (None, 27, 5650) output shape: (None, 27, 27)
2_lstm_layer: input shape: (None, 27, 27) output shape: (None, 27)
2_dropout_layer: input shape: (None, 27) output shape: (None, 27)
dense1_layer: input shape: (None, 27) output shape: (None, 512)
elu_3: input shape: (None, 512) output shape: (None, 512)
3_dropout_layer: input shape: (None, 512) output shape: (None, 512)
softmax_layer: input shape: (None, 512) output shape: (None, 18)


In [ ]:
batchSize = 200
train_epoches = 50

model.fit(reshaped_train,train_labels,validation_data=(reshaped_val,val_labels),epochs=train_epoches,batch_size=batchSize,verbose=1)

print('Calculating score.. ')
score = model.evaluate(reshaped_test,test_labels,verbose=1)
print(score)

train_epoches = 20
all_train = np.concatenate((reshaped_train, reshaped_val))
all_labels = np.concatenate((train_labels, val_labels))
model.fit(all_train,all_labels,validation_data=(reshaped_test,test_labels),epochs=train_epoches,batch_size=batchSize,verbose=1)

print('Calculating score.. ')
score = model.evaluate(reshaped_test,test_labels,verbose=1)
print(score)
model.save('taskB2_all_Subjects_CNN_LSTM_overall.h5')

Train on 49696 samples, validate on 6089 samples
Epoch 1/50
49696/49696 [==============================] - 49s 994us/step - loss: 1.2194 - acc: 0.6917 - val_loss: 0.7439 - val_acc: 0.8343
Epoch 2/50
49696/49696 [==============================] - 44s 882us/step - loss: 0.7610 - acc: 0.7592 - val_loss: 0.5249 - val_acc: 0.8394
Epoch 3/50
49696/49696 [==============================] - 44s 879us/step - loss: 0.5995 - acc: 0.7947 - val_loss: 0.5139 - val_acc: 0.8594
Epoch 4/50
49696/49696 [==============================] - 43s 873us/step - loss: 0.5246 - acc: 0.8145 - val_loss: 0.4754 - val_acc: 0.8602
Epoch 5/50
49696/49696 [==============================] - 43s 871us/step - loss: 0.4812 - acc: 0.8269 - val_loss: 0.4998 - val_acc: 0.8568
Epoch 6/50
49696/49696 [==============================] - 44s 877us/step - loss: 0.4296 - acc: 0.8421 - val_loss: 0.4870 - val_acc: 0.8622
Epoch 7/50
49696/49696 [==============================] - 43s 875us/step - loss: 0.4092 - acc: 0.8495 - val_loss: 0.4

In [ ]:
predictions = model.predict(reshaped_test)

In [ ]:
# F1-score measure
from sklearn.metrics import f1_score
num_classes = 18
class_predictions = []
class_true = []
tot_labels = 0.0
count = 0.0
for pair in zip(predictions, test_labels):
    class_predictions.append(np.argmax(pair[0]))
    class_true.append(np.argmax(pair[1]))
    if np.argmax(pair[0]) == np.argmax(pair[1]):
        count += 1.0
    tot_labels += 1.0
    
print('Standard accuracy is ' + str(count/tot_labels))    

unique, counts = np.unique(class_true, return_counts=True)
counted_labels = dict(zip(unique, counts))
f1_scores = f1_score(class_predictions, class_true, average=None)

tot_f1_score = 0.0
weights_sum = 0.0
for i in range(num_classes):
    labels_class_i = counted_labels[i]
    weight_i = labels_class_i / tot_labels
    weights_sum += weight_i
    tot_f1_score += f1_scores[i]*weight_i
    print(str(i) + ' ' + str(weight_i) + ' ' + str(f1_scores[i]))

    
print('The weigths sum is ' + str(weights_sum))
print('The computed f1-score is {}'.format(tot_f1_score))
print('The f1-score with sklearn function is {}'.format(f1_score(class_true, class_predictions, average='weighted')))
print('Average f1-score is {}'.format(np.mean(f1_scores)))

In [ ]:
from keras.models import Model
from sklearn.decomposition import PCA

layer_name = 'dense_layer'
inter_layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)

network_output = inter_layer_model.predict(all_train)

# PCA with certain number of principal components
pca = PCA(n_components=40)
pca.fit(network_output)
new_output = pca.transform(network_output)

In [ ]:
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

svm_labels = np.argmax(all_labels, axis=1)

# train a one-vs-one multi-class support vector machine
clf = svm.SVC(decision_function_shape='ovo')
clf.fit(new_output, svm_labels)

# predict test data
svm_pred = clf.predict(pca.transform(inter_layer_model.predict(reshaped_test)))

# measure accuracy and f1-score
num = 0.0
den = 0.0
new_test_labels = np.argmax(test_labels, axis=1)
for pair in zip(svm_pred, new_test_labels):
    if pair[0] == pair[1]:
        num += 1.0
    
    den += 1.0

scores = cross_val_score(clf, pca.transform(inter_layer_model.predict(reshaped_test)),new_test_labels, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

print('Test accuracy is: {}'.format(num / den))
f1_scores = f1_score(svm_pred, new_test_labels, average=None)
print('The f1-score with sklearn function is {}'.format(f1_score(new_test_labels, svm_pred, average='weighted')))
print('Average f1-score is {}'.format(np.mean(f1_scores)))

In [ ]:
from sklearn.model_selection import cross_val_predict

predicted = cross_val_predict(clf, pca.transform(inter_layer_model.predict(reshaped_test)),new_test_labels, cv=5)

# measure accuracy and f1-score
num = 0.0
den = 0.0
new_test_labels = np.argmax(test_labels, axis=1)
for pair in zip(predicted, new_test_labels):
    if pair[0] == pair[1]:
        num += 1.0
    
    den += 1.0

print('Test accuracy is: {}'.format(num / den))
f1_scores = f1_score(predicted, new_test_labels, average=None)
print('The f1-score with sklearn function is {}'.format(f1_score(new_test_labels, predicted, average='weighted')))
print('Average f1-score is {}'.format(np.mean(f1_scores)))

In [ ]:
pred_df = pd.DataFrame(predictions)
pred_df.to_csv('preds_test.csv', header=False, index=False)

In [ ]:
true_df = pd.DataFrame(testY)
true_df.to_csv('true_test.csv', header=False, index=False)